<a href="https://colab.research.google.com/github/greapis/Tensorflow_Python/blob/main/Tensorflow_AI_LS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Library

 - Keras에서 일부 전처리된 형태의 IMDB Data 제공
 - Keras - sklearn이 서로 호환되지 않기 때문에 "KerasClassifier" 필요 : 이 부분은 Tensorflow가 업데이트되면서 변동되어서 확인 필요

In [17]:
# Importing Libraries
  # Keras abd Sklearn libraries
    # Importing KerasClassifier Library : Keras vs Sklearn 호환성 이슈 해결


# IMDB Datasets 불러오기 from Keras
from tensorflow.keras.datasets import imdb

# 리뷰 길이 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Making Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

# Keras model - (래핑) - sklearn이 이해할 수 있는 형태로 변환
# Install scikeras package : "scikeras" 설치 필요
!pip install scikeras
from scikeras.wrappers import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier  : 최신 TensorFlow(특히 2.12 이상)에서는 tensorflow.keras.wrappers.scikit_learn 모듈이 더 이상 제공되지 않습니다.

# Importing GreidSearch
from sklearn.model_selection import GridSearchCV

# Caculating Accuracy
from sklearn.metrics import accuracy_score

# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier :


*  최신 TensorFlow(특히 2.12 이상)에서는 tensorflow.keras.wrappers.scikit_learn 모듈이 더 이상 제공되지 않습니다.
*  과거에는 Keras가 scikit-learn과의 호환성을 위해 이 모듈을 제공했지만, 이제는 공식적으로 지원을 중단했으며, 이 기능은 별도의 패키지로 분리되었습니다



*   scikeras 사용 : 대안으로 scikeras라는 패키지를 사용할 수 있습니다.
다음처럼 임포트하면 됩니다:
*   from scikeras.wrappers import KerasClassifier 또는 from scikeras.wrappers import KerasRegressor






Step 2. Data 전처리

 - 리뷰 데이터 형태를 '숫자'로 변환 : ex[1, 5, 35, 6,... ,532]
 - label : Positive(1), Negative(0)

In [18]:
# Loading datas and 전처리


# words dictionary : 상위 10000개 까지 단어만 사용(커트라인)
# 문장 길이(maxien) : 256단어 길이로 제한 통일
num_words = 10000
maxlen = 256


# Seperating Learning Data // Test Data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)


# Making equal the sentence length
X_train = pad_sequences(X_train, maxlen=maxlen) # Corrected keyword argument
X_test = pad_sequences(X_test, maxlen=maxlen) # Corrected keyword argument

Step 3. Learning and Validation

 - 모델을 함수로 정의
 - 기존과 달리 모델을 함수로 정의(def로 함수 정의 사용)
 - Grid Search에서 재사용하기 위해 함수로 정의 사용
 - 함수로 3가지 Layer 정의 사용(Embedding Layer, LSTM Layer. Fully Connected Layer)

In [19]:
# Model 구축 // Function definition


# Model을 function으로 정의 : hyperparameter를 튜닝하기 위해 // Gridsearch에서 재사용

def build_model(unit = 64):
    model = Sequential()
    # embedding layer(단어(text)를 숫자(scalar)로 그리고 다시 숫자(128차원 Vector로))
    model.add(Embedding(input_dim=num_words, output_dim=128, input_length=maxlen))
    # LSTM Layer
    model.add(LSTM(units))
    # Fully Connected Layer (Number of Neuron = 1(결과값이 1개 필요해서 : 1 or 0),  Sigmoid activation function : 0~1사이의 값으로 변환)
    model.add(Dense(1, activation='sigmoid'))
    # 최적화 : adam사용, 손실함수 : binary_crossentropy, 평가지표 : accuracy
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [20]:
# Learning and Validation v2


# Keras model을 scikit-learn 포맷으로 래핑? : 최신 TensorFlow(특히 2.12 이상)에서는 tensorflow.keras.wrappers.scikit_learn 모듈이 더 이상 제공 안됨
#    설치된 "scikeras" library 사용(from scikeras.wrappers import KerasClassifier)

model = KerasClassifier(build_fn=build_model, verbose=1)
  #  verbose = 0 : 훈련 과정 안 보임 // verbose = 1 : 훈련 과정 표시

# Defining Hyperparameter Grid
param_grid = {
    'units': [32, 64],
    'batch_size': [16, 32],
    'epochs': [2,3,4]
}


# CV(Cross Validation) + Tuning
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
    # model을 esimator에 집어 넣고 실험, "cv=3"은 3-Folde Cross Validation 실행
grid_result = grid.fit(X_train, y_train)


# Optimal Parameter 출력 (최적의 hypaerparameter 조합 실행)
print("Best Parameters:", grid_result.best_params_)
print("Best Cross-Validation Accuracy:", grid_result.best_score_)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

Step 4. Test

 - Best Model Final Test
 - 검증을 통해서 채택한 Optimal Hyperparameter model에 test data 적용 : "Accuracy" 산출